In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
import piplite
await piplite.install("palettable")
await piplite.install("seaborn")
import VisualizeNN as VisNN
import pyodide
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
sns.set(rc={'figure.figsize':(11.7,8.27)})

## 1. 座標軸データにおける学習

In [ ]:
data = pd.read_csv(pyodide.http.open_url("https://raw.githubusercontent.com/ChungWookyung/kg-jupyterlite-data-analysis/main/content/DATA/xydata.csv"))
data.head()

データが何を意味しているのか、グラフで見る（散布図）

In [ ]:
sns.scatterplot(data = data, x = "x", y = "y", hue = "label")

ラベルが１の場合、真ん中の円の中の点であり、０はそれ以外の点であることが分かる

### 1-1 学習データとテストデータを分ける

In [ ]:
train = data.sample(frac=0.8, random_state=1234)
test = data.drop(train.index)
train.head()

In [ ]:
test.head()

800個の学習データtrainと200個のテストデータtestに分けた

In [ ]:
sns.scatterplot(data = train, x = "x", y = "y", hue = "label")

学習データの散布図

In [ ]:
sns.scatterplot(data = test, x = "x", y = "y", hue = "label")

テストデータの散布図

### 1-2 ニューラルネットワークの構築

In [ ]:
X_train = train.loc[:,["x","y"]].values
Y_train = train.loc[:,["label"]].values
Y_train = np.ravel(Y_train)

まずは、ニューラルネットワークモデルを学習させるためのXとYデータを分けておく
- X_train : 座標軸の学習データ
- Y_train : ラベルの学習データ

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(8,4,), alpha=0.5, max_iter=5000, random_state=1234)
model = model.fit(X_train, Y_train)

１階層に8つのニューロンがあり、２階層に４つのニューロンがあるモデルを構築し、学習させた

In [ ]:
model.score(X_train, Y_train)

学習データにおける正確度は98%である

In [ ]:
X_test = test.loc[:,["x","y"]].values
Y_test = test.loc[:,["label"]].values
Y_test = np.ravel(Y_test)
model.score(X_test, Y_test)

テストデータにおいても同じく98%の正確度を見せている

In [ ]:
model_data = pd.DataFrame({"x":train.x, "y":train.y, "label":model.predict(X_train)})
sns.scatterplot(data = model_data, x = "x", y = "y", hue = "label")

モデルが予測した学習データにおけるラベルは上のようになる

In [ ]:
xx, yy = np.meshgrid(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01))
Z = model.predict_proba(np.column_stack([xx.ravel(), yy.ravel()]))[:, 1]
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.bwr, alpha=0.8)

モデルが予測する範囲

## 1-3 ニューラルネットワークモデルの図式化

In [ ]:
network_structure = np.hstack(([X_train.shape[1]], np.asarray(model.hidden_layer_sizes), [1]))
network=VisNN.DrawNN(network_structure)
network.draw()

モデルの各ニューロンは上のようにつながっている

In [ ]:
network=VisNN.DrawNN(network_structure, model.coefs_)
network.draw()

学習されたニューロンの間の連結を図式化した

## 2 鉱石のデータセット

In [ ]:
data = pd.read_csv(pyodide.http.open_url("https://raw.githubusercontent.com/ChungWookyung/kg-jupyterlite-data-analysis/main/content/DATA/sonar.csv"))
data.head()

データは６０個のセンサーで測った石の形の情報(sensor1~60)とその石が鉱石かただの石かを表すラベル(label)で構成されている

In [ ]:
np.unique(data.label)

鉱石だったら、ラベルは"M"となり、石だったら、ラベルは"R"となる

In [ ]:
data.label = data.label.replace({"M":1, "R":0})

データのラベルを鉱石だったら1、石だったら0に書き換えた

In [ ]:
sns.scatterplot(data = data, x = "sensor1", y = "sensor2", hue = "label")

上の散布図のように、各センサーごとの鉱石と石の散布図を描くことができるが、60個もあるので、人がいちいち判断しにくい

### 2-1 学習データとテストデータを分ける

In [ ]:
train = data.sample(frac=0.8, random_state=1234)
test = data.drop(train.index)
train.head()

In [ ]:
test.head()

166個の学習データtrainと42個のテストデータtestに分けた

In [ ]:
fig, ax = plt.subplots(5,5,figsize=(12,12))

for i in range(5):
    for j in range(5):
        sensor1 = "sensor" + str(i+1)
        sensor2 = "sensor" + str(j+10)
        sns.scatterplot(data=train, x=sensor1, y=sensor2, hue = "label", ax=ax[i][j])
        ax[i][j].legend([],[], frameon=False)

学習データにおける各センサーごとの散布図(オレンジ色：鉱石、青色：ただの石)

In [ ]:
fig, ax = plt.subplots(5,5,figsize=(12,12))

for i in range(5):
    for j in range(5):
        sensor1 = "sensor" + str(i+1)
        sensor2 = "sensor" + str(j+10)
        sns.scatterplot(data=test, x=sensor1, y=sensor2, hue = "label", ax=ax[i][j])
        ax[i][j].legend([],[], frameon=False)

テストデータにおける各センサーごとの散布図(オレンジ色：鉱石、青色：ただの石)

### 2-2 ニューラルネットワークの構築

In [ ]:
X_train = train.iloc[:,np.arange(60)].values
Y_train = train.loc[:,["label"]].values
Y_train = np.ravel(Y_train)

まずは、ニューラルネットワークモデルを学習させるためのXとYデータを分けておく

- X_train : センサーの学習データ
- Y_train : ラベルの学習データ

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(32,32,), alpha=0.5, max_iter=5000, random_state=1234)
model = model.fit(X_train, Y_train)

１階層に32個のニューロンがあり、２階層に32個のニューロンがあるモデルを構築し、学習させた

In [ ]:
model.score(X_train, Y_train)

学習データにおける正確度は100%である

In [ ]:
X_test = test.iloc[:,np.arange(60)].values
Y_test = test.loc[:,["label"]].values
Y_test = np.ravel(Y_test)
model.score(X_test, Y_test)

テストデータにおいて83%の正確度を見せている

強いてモデルを図式化するとすれば、次のようなものになる
![](https://raw.githubusercontent.com/ChungWookyung/kg-jupyterlite-data-analysis/main/content/DA23/nn.svg)

## 3 ペンギンのデータセット

In [ ]:
data = pd.read_csv(pyodide.http.open_url("https://raw.githubusercontent.com/ChungWookyung/kg-jupyterlite-data-analysis/main/content/DATA/penguin.csv"))
data.head()

実際に測られたペンギンのデータを読み込んだ

- species : どの種のペンギンなのか
- island : どの島に住んでいるのか
- bill_length_mm : くちばしの長さ
- bill_depth_mm : くちばしの深さ
- flipper_length_mm : 羽の長さ
- body_mass_g : 体重
- sex : 性別
- year : 測った年度

In [ ]:
sns.scatterplot(data = data, x = "bill_length_mm", y = "flipper_length_mm", hue = "species")

くちばしと羽の長さの散布図

In [ ]:
data = pd.read_csv(pyodide.http.open_url("https://raw.githubusercontent.com/ChungWookyung/kg-jupyterlite-data-analysis/main/content/DATA/penguin.csv"))
data = data.drop("year",axis=1)
data.sex = data.sex.replace({"female":1,"male":0})
data = pd.get_dummies(data, columns=["island","species"])
data.bill_length_mm = (data.bill_length_mm - data.bill_length_mm.mean())/data.bill_length_mm.std()
data.bill_depth_mm = (data.bill_depth_mm - data.bill_depth_mm.mean())/data.bill_depth_mm.std()
data.flipper_length_mm = (data.flipper_length_mm - data.flipper_length_mm.mean())/data.flipper_length_mm.std()
data.body_mass_g = (data.body_mass_g - data.body_mass_g.mean())/data.body_mass_g.std()
data.head()

データを学習しやすいように書き直す
- ダミー変数に書き換える作業
- -1から1に収まるように正規化する作業

### 3-1 学習データとテストデータを分ける

In [ ]:
train = data.sample(frac=0.8, random_state=1234)
test = data.drop(train.index)
train.head()

In [ ]:
test.head()

266個の学習データtrainと67個のテストデータtestに分けた

### 3-2 ニューラルネットワークの構築

In [ ]:
X_train = train.iloc[:,np.arange(8)].values
Y_train = train.loc[:,["species_Adelie","species_Chinstrap","species_Gentoo"]].values

まずは、ニューラルネットワークモデルを学習させるためのXとYデータを分けておく

- X_train : ペンギンの基本情報の学習データ
- Y_train : ペンギンの種の学習データ

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(16,8,), alpha=0.5, max_iter=5000, random_state=1234)
model = model.fit(X_train, Y_train)

１階層に16個のニューロンがあり、２階層に8個のニューロンがあるモデルを構築し、学習させた

In [ ]:
model.score(X_train, Y_train)

学習データにおける正確度は100%である

In [ ]:
X_test = test.iloc[:,np.arange(8)].values
Y_test = test.loc[:,["species_Adelie","species_Chinstrap","species_Gentoo"]].values
model.score(X_test, Y_test)

テストデータにおける正確度も100%である

## 3-3 ニューラルネットワークモデルの図式化

In [ ]:
network_structure = np.hstack(([X_train.shape[1]], np.asarray(model.hidden_layer_sizes), [3]))
network=VisNN.DrawNN(network_structure)
network.draw()

モデルの各ニューロンは上のようにつながっている

In [ ]:
network=VisNN.DrawNN(network_structure, model.coefs_)
network.draw()

学習されたニューロンの連結を図式化した